# Build the small dataset for tests

In [46]:
import pandas as pd
import gffutils
import os
from collections import defaultdict
pd.set_option('max_colwidth', 800)
pd.set_option('display.width', 1000)


In [47]:
infile = '/home/bay001/projects/codebase/annotator/annotator/test/data/gencode.v19.annotation.gtf'
outfile = '/home/bay001/projects/codebase/annotator/annotator/test/data/gencode.v19.annotation.chr11.70M-71M.gtf'
db_file = '/home/bay001/projects/codebase/annotator/annotator/test/data/gencode.v19.annotation.chr11.70M-71M.gtf.db'

def build_db(annotation_file, db_file, force=True, disable_infer_genes=True, disable_infer_transcripts=True):
    db = gffutils.create_db(
        annotation_file, dbfn=db_file, force=force, # change to True if we need to create a new db
        keep_order=True, merge_strategy='merge', sort_attribute_values=True,
        disable_infer_genes=disable_infer_genes,
        disable_infer_transcripts=disable_infer_transcripts
    )

In [9]:
# build_db(annotation_file, db_file, disable_infer_genes=False, disable_infer_transcripts=False)
infile = '/home/bay001/projects/codebase/annotator/annotator/test/data/gencode.v19.annotation.gtf'
outfile = '/home/bay001/projects/codebase/annotator/annotator/test/data/gencode.v19.annotation.chr11.70M-71M.gtf'

o = open(outfile,'w')
with open(infile,'r') as f:
    for line in f:
        if not line.startswith('#'):
            line = line.split('\t')
            if line[0] == 'chr11' and int(line[3]) > 69924407 and int(line[4]) < 71000000:
                o.write('\t'.join(line))

o.close()

In [10]:
build_db(outfile, db_file, disable_infer_genes=False, disable_infer_transcripts=False)

# Build the test database for prox and distal intron testing
- mostly using this notebook for peeking into the gtf file and seeing what constitutes a gene

In [55]:
# infile = '/home/bay001/projects/codebase/annotator/annotator/test/data/proxdist_test.gtf'
infile = '/projects/ps-yeolab/genomes/hg19/gencode_v19/gencode.v19.annotation.gtf'
df = pd.read_table(
    infile, names=['seqname','src','feature','start','end','score','strand','frame','attribute'], comment='#'
)
df.head()

,seqname,src,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,11869,14412,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENSG00000223972.4""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""pseudogene""; transcript_status ""KNOWN""; transcript_name ""DDX11L1""; level 2; havana_gene ""OTTHUMG00000000961.2"";"
1,chr1,HAVANA,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";"
2,chr1,HAVANA,exon,11869,12227,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; exon_number 1; exon_id ""ENSE00002234944.1""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";"
3,chr1,HAVANA,exon,12613,12721,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; exon_number 2; exon_id ""ENSE00003582793.1""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";"
4,chr1,HAVANA,exon,13221,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""ENST00000456328.2""; gene_type ""pseudogene""; gene_status ""KNOWN""; gene_name ""DDX11L1""; transcript_type ""processed_transcript""; transcript_status ""KNOWN""; transcript_name ""DDX11L1-002""; exon_number 3; exon_id ""ENSE00002312635.1""; level 2; tag ""basic""; havana_gene ""OTTHUMG00000000961.2""; havana_transcript ""OTTHUMT00000362751.1"";"


In [49]:
db_file = '/home/bay001/projects/codebase/annotator/annotator/test/data/proxdist_test.gtf.db'
build_db(infile, db_file, disable_infer_genes=False, disable_infer_transcripts=False)

/home/bay001/anaconda2/envs/brian/lib/python2.7/site-packages/gffutils/create.py:714: UserWarning: It appears you have a gene feature in your GTF file. You may want to use the `disable_infer_genes` option to speed up database creation
  "It appears you have a gene feature in your GTF "
/home/bay001/anaconda2/envs/brian/lib/python2.7/site-packages/gffutils/create.py:705: UserWarning: It appears you have a transcript feature in your GTF file. You may want to use the `disable_infer_transcripts` option to speed up database creation
  "It appears you have a transcript feature in your GTF "


In [56]:
df['coding'] = df['attribute'].str.extract("gene_type \"([\w\d\.]+)\"")
df['coding'].value_counts()

/home/bay001/anaconda2/envs/brian/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


protein_coding              2402832
pseudogene                    70989
lincRNA                       51893
antisense                     41470
processed_transcript          13567
miRNA                          9165
misc_RNA                       6102
snRNA                          5748
snoRNA                         4371
polymorphic_pseudogene         3515
sense_intronic                 3175
rRNA                           1581
sense_overlapping              1352
IG_V_gene                      1123
TR_V_gene                       763
IG_V_pseudogene                 681
TR_J_gene                       300
IG_C_gene                       185
IG_D_gene                       152
3prime_overlapping_ncrna        100
TR_V_pseudogene                  99
IG_J_gene                        82
Mt_tRNA                          66
TR_C_gene                        58
IG_C_pseudogene                  36
TR_J_pseudogene                  12
TR_D_gene                        12
IG_J_pseudogene             